https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

In [1]:
# prepare data for lstm
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [ ]:
# https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df =  DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
# load dataset
dataset = read_csv('SeoulBikeData2.csv', encoding= 'unicode_escape', header=0, index_col=0)
values = dataset.values

In [ ]:
# encode the categorical variables into integers
'''label_encoder = LabelEncoder()
one_hot = OneHotEncoder()
values[:,10] = label_encoder.fit_transform(values[:,10])
values[:,11] = label_encoder.fit_transform(values[:,11])
values[:,12] = label_encoder.fit_transform(values[:,12])'''

In [ ]:
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [ ]:
# frame as supervised learning
'''reframed = series_to_supervised(scaled_data, past_days_to_consider, number_of_predictions_into_the_future)
# drop columns we don't want to predict
reframed.drop(reframed.columns[[number,of,columns]], axis=1, inplace=True)
print(reframed.head())'''

# Test 3:
<h1>24 hours
    hour1: all parameters,
    <p>hour23 to hour24: bike_count
</h1>

In [ ]:
# frame as supervised learning
hours_to_consider = 23
reframed = series_to_supervised(scaled, hours_to_consider, 1)
# drop columns we don't want to predict
columns_to_keep = list()
for i in range(10,reframed.shape[1]+1,9):
    columns_to_keep.append(i)
x = 11
n_cols = [i for i in range (10, 216+1)]
for i in columns_to_keep:
    n_cols.remove(i)
n_cols = [x - 1 for x in n_cols]
reframed.drop(reframed.columns[[n_cols]], axis=1, inplace=True)
print(reframed.columns)
print(reframed.head())

In [ ]:
# split into train and test sets
values = reframed.values
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 1.0 - train_ratio - val_ratio
train_days = 365 * train_ratio

n_train_hours = train_days * 24

train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# fit network
history = model.fit(train_X, train_y, epochs=5, batch_size=60, validation_data=(test_X, test_y), verbose=2, shuffle=False)

# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()